In [1]:
#/usr/bin/python
# -*- coding: utf-8 -*-

import os
import sys
import json
import keys
import glob
import re
from requests_oauthlib import OAuth1Session

In [2]:
#API認証情報
CK = keys.CK
CS = keys.CS
AT = keys.AT
ATS = keys.ATS
twitter = OAuth1Session(CK,CS,AT,ATS)

#API取得
search = "https://api.twitter.com/1.1/search/tweets.json?tweet_mode=extended"

# ツイートの取得設定
num = 10
keyword = '#VRChatワールド紹介' + ' -filter:retweets'
params = {'q' : keyword, 'count' : num}
req = twitter.get(search, params = params)

In [3]:
# 画像を保存するディレクトリ
image_dir_path = os.path.normpath(os.path.join(os.path.dirname(os.path.abspath("__file__")), "../images"))
path = os.path.exists(image_dir_path)
if not path:
    os.mkdir(image_dir_path)

In [4]:
def generate_image_num():
    image_num_max = 50
    image_path_list = sorted(glob.glob(image_dir_path + '/[0-9][0-9][0-9].jpg'))
    image_name_list = [os.path.basename(p) for p in sorted(image_path_list) if os.path.isfile(p)]

    if len(image_name_list) == 0:
        image_num = 0
    else:
        image_num_last = int(re.sub(r"\D", "", image_name_list[-1]))
        if image_num_last < image_num_max:
            image_num = image_num_last + 1
        else:
            # 一番古いファイルパス
            image_path_oldest = sorted(image_path_list, key=lambda f: os.stat(f).st_mtime, reverse=True)[-1]
            print(image_path_oldest)
            image_num = int(re.sub(r"\D", "", image_path_oldest))
    
    return image_num

In [5]:
if req.status_code == 200:
    # ツイートの画像を取得
    tweets = json.loads(req.content)
    for tweet in tweets['statuses']:
        if 'media' in tweet['extended_entities']:
            for media in tweet['extended_entities']['media']:
                media_urls = media['media_url']
                downloads = twitter.get(media_urls).content
                # 画像を保存
                image_num = generate_image_num()
                file_name = '{0:03d}'.format(image_num) + '.jpg'
                file_path = image_dir_path + '/' + file_name
                images = open(file_path, 'wb')
                images.write(downloads)
                images.close()

else:
    print("ERROR: %d" % req.status_code)

/Users/kuwagata/repos/utsutsuroi_twitter/images/000.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/001.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/002.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/003.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/004.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/005.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/006.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/007.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/008.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/009.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/010.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/011.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/012.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/013.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/014.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/015.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/016.jpg
/Users/kuwagata/repos/utsutsuroi_twitter/images/

In [6]:
tweets['statuses'][0]['full_text']

'No.371\nワールド名：-Blue-\n製作者：Luche-\nQuest対応：非対応\n最大接続数：15人\nLink：https://t.co/5Ir4j4Nxzn\n\n#VRChat_world写真一枚勝負\n#VRChat_world紹介 \n#VRChatワールド紹介 https://t.co/P5pXuhnXzO'